In [92]:
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [93]:
import os
root_dir = '..'
games_fn = os.path.join(root_dir, '2019-05-02.csv')
details_fn = os.path.join(root_dir, 'games_detailed_info.csv')
reviews_fn = os.path.join(root_dir, 'bgg-13m-reviews.csv')
placeholder_img_url = 'https://via.placeholder.com/150x150?text=No+Image'

In [94]:
def print_nan_stat(df):
    for c in df.columns.values:
        vals = df[[c]]
        num_na = int(vals.isna().sum())
        if num_na > 0:
            print('{}: {} NaN values'.format(c, num_na))

In [95]:
games = pd.read_csv(games_fn)
#games = games.set_index('ID') # don't do this
games = games.drop(columns=['Bayes average', 'URL'])
games['Thumbnail'] = games['Thumbnail'].fillna(placeholder_img_url)

# GET /boardgame/ 
#  display:  Rank, Thumbnail, Name, Year 
#  optional: (Average, Users rated)
#  argument for other URLs: ID
#  (Rank can be inferred from the Average so you could argue its useless)
#  (list already sorted by Rank, ascending)

# [TEST ONLY]
# example iteration for generating / paginating the webpage
for i, row in games.head(3).iterrows():
    print("{}. {} ({}) -- {}".format(row['Rank'], row['Name'], row['Year'], i))
#print("\r\n{}".format(games.loc[3439].tolist()))  # needs set_index to be applied
print("\r\n{}".format(games[games['ID']==3439].iloc[0].tolist()))  # image URL originally NaN

1. Gloomhaven (2017) -- 0
2. Pandemic Legacy: Season 1 (2015) -- 1
3. Through the Ages: A New Story of Civilization (2015) -- 2

[3439, 'HeroClix', 2002, 1922, 6.5, 2261, 'https://via.placeholder.com/150x150?text=No+Image']


In [96]:
print("NaN stats below:") # [TEST ONLY]
print_nan_stat(games)

NaN stats below:


In [97]:
# get JSON dict of N entries (for pagination, or None for all) starting from Nth entry
# retval: list of dicts, where each dict is 1 row
def get_json_entries(df, start_pos = None, num_rows = None, to_json=True):
    if start_pos == None:
        start_pos = 0
    end_pos = len(df.index) if (num_rows == None) else min(start_pos + num_rows, len(df.index))
    selected = df.iloc[start_pos : end_pos]
    return selected.to_json(orient='records') if (to_json) else selected.to_dict(orient='records')



# [TEST ONLY]
get_json_entries(games,1,2,False)    # note: rank starts from 1, start_pos starts from 0

[{'ID': 161936,
  'Name': 'Pandemic Legacy: Season 1',
  'Year': 2015,
  'Rank': 2,
  'Average': 8.64,
  'Users rated': 30921,
  'Thumbnail': 'https://cf.geekdo-images.com/micro/img/KbAKyhbG4qab4r-A_pBjUGvgal0=/fit-in/64x64/pic2452831.png'},
 {'ID': 182028,
  'Name': 'Through the Ages: A New Story of Civilization',
  'Year': 2015,
  'Rank': 3,
  'Average': 8.53,
  'Users rated': 15551,
  'Thumbnail': 'https://cf.geekdo-images.com/micro/img/APvZ_BYOt4ElpIXVl7i6wUp2BvM=/fit-in/64x64/pic2663291.jpg'}]

In [107]:
details = pd.read_csv(details_fn)
#print(details.columns)
details = details[['id','primary','boardgamepublisher','boardgamecategory','minplayers','maxplayers','minage','minplaytime','description','boardgameexpansion','boardgamemechanic','image']]
details.columns = ['ID','Name',   'Publisher',         'Category',         'MinPlayers','MaxPlayers','MinAge','MinPlaytime','Description','Expansion',         'Mechanic',         'Thumbnail']

details[details['Name']=='Dominion']
#details[details['Name']=='Dominion'].to_dict('record')
#games.sort_values(by=['ID']).head(13)

,ID,Name,Publisher,Category,MinPlayers,MaxPlayers,MinAge,MinPlaytime,Description,Expansion,Mechanic,Thumbnail
4,36218,Dominion,"['Rio Grande Games', '999 Games', 'Albi', 'Bar...","['Card Game', 'Medieval']",2,4,13,30,(from the back of the box:)&#10;&#10;&quot;You...,['Animals expansion (mini fan expansion for Do...,"['Card Drafting', 'Deck / Pool Building', 'Han...",https://cf.geekdo-images.com/original/img/oN8C...


,ID,Name,Year,Rank,Average,Users rated,Thumbnail
222,1,Die Macher,1986,223,7.63,4777,https://cf.geekdo-images.com/micro/img/JFMB8OR...
3094,2,Dragonmaster,1981,3095,6.62,523,https://cf.geekdo-images.com/micro/img/GjYccOg...
187,3,Samurai,1998,188,7.44,13455,https://cf.geekdo-images.com/micro/img/4XUy5Qx...
4032,4,Tal der Könige,1992,4033,6.61,324,https://cf.geekdo-images.com/micro/img/x2IMJSP...
223,5,Acquire,1964,224,7.35,16946,https://cf.geekdo-images.com/micro/img/SR4x4Yj...
8207,6,Mare Mediterraneum,1989,8208,6.53,79,https://cf.geekdo-images.com/micro/img/ldvo1mb...
1629,7,Cathedral,1978,1630,6.51,3002,https://cf.geekdo-images.com/micro/img/K7inVaE...
6926,8,Lords of Creation,1993,6927,6.15,190,https://cf.geekdo-images.com/micro/img/NG_cQx4...
2211,9,El Caballero,1998,2212,6.48,1336,https://cf.geekdo-images.com/micro/img/nxTDdLh...
863,10,Elfenland,1998,864,6.73,7383,https://cf.geekdo-images.com/micro/img/BQiVfOs...
